# Heart rate analysis

This tutorial shows how to extract heart rate estimates using photoplethysmography (PPG) data and accelerometer data. 

## Load example data

Example PPG + accelerometer (from IMU files) data from a participant of the Personalized Parkinson Project is loaded. The data was prepared as explained in `data_preparation.ipynb`. The prepared IMU data contains both accelerometer and gyroscope data, but only accelerometer is used in this pipeline alongside the PPG data.

In [ ]:
from pathlib import Path
from paradigma.util import load_tsdf_dataframe

path_to_data = Path('../../../tests/data')
path_to_prepared_data = path_to_data / '1.prepared_data'


## Step 1: Preprocess data

The collected sensor data is collected at a fixed sampling frequency, but the sampling rate is not uniform, causing variation in time differences between timestamps. The preprocess_ppg_data function therefore resamples the values of both PPG and accelerometer data using uniformly distributed timestamps. After this, a bandpass filter (butterworth, 4th-order, cut-off frequencies: [0.4, 3.5]) is applied to the PPG signal and a high-pass (butterworth, 4th-order, cut-off: 0.2 Hz) filter is applied to the accelerometer. 

In [ ]:
df_ppg_proc, df_acc_proc = preprocess_ppg_data(metadatas_ppg[0], metadatas_imu[0], path_to_preprocessed_data, ppg_config, imu_config)

## Step 2: Extract signal quality features

## Step 3: Signal quality classification

## Step 4: Heart rate estimation

## Step 5: Heart rate aggregation